In [1]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 18.5 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
# Transformers ve peft kütüphaneleri
!pip install transformers accelerate peft datasets

In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/deneme1.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)

# PyTorch formatına çevir
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [ ]:
!pip install -q --upgrade transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 17.2 MB/s eta 0:00:00


In [7]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model # PEFT için gerekli kütüphaneleri ekle

# LoRA yapılandırması
lora_config = LoraConfig(
    r=8,  # LoRA dikkat katmanları için rank (rank)
    lora_alpha=16,  # LoRA ölçeklendirme faktörü
    target_modules=["q_proj", "v_proj"], # LoRA uygulanacak katmanlar
    lora_dropout=0.1,  # LoRA katmanlarında dropout oranı
    bias="none",  # Bias uygulanmayacak
    task_type="CAUSAL_LM"  # Görev türü: Causal Language Modeling
)

# Modeli PEFT modeli olarak hazırla
model = get_peft_model(model, lora_config)

# Eğitilebilir parametrelerin sayısını yazdır
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./lora_mistral",
    per_device_train_batch_size=1,  # Colab GPU hafızasına göre
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,   # batch simülasyonu
    learning_rate=3e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    save_total_limit=2,
    remove_unused_columns=False
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator # Pass the data collator here
)

trainer.train()

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


Step,Training Loss,Validation Loss
50,1.376196,1.544211


TrainOutput(global_step=54, training_loss=1.7888555879946109, metrics={'train_runtime': 368.556, 'train_samples_per_second': 0.586, 'train_steps_per_second': 0.147, 'total_flos': 4720554028302336.0, 'train_loss': 1.7888555879946109, 'epoch': 3.0})

In [17]:
def evaluate_answer(question, answer):
    prompt = f"""<s>[INST]
You are a strict senior HR evaluator.

Score the candidate using this rubric:

Clarity (0-2)
Structure (0-2)
Relevance (0-2)
Quality of Example (0-2)
Professional Communication (0-2)

Add the points and give TOTAL score out of 10.

Be strict. Do NOT give high score unless clearly excellent.

Return ONLY in this JSON format:

{{
  "clarity": ,
  "structure": ,
  "relevance": ,
  "example_quality": ,
  "professionalism": ,
  "total_score": ,
  "strengths": "",
  "weaknesses": ""
}}

Question:
{question}

Candidate Answer:
{answer}
[/INST]"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        temperature=0.1
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("[/INST]")[-1].strip()

In [19]:
question = "Tell me about a time you had to learn something quickly."

answer = """
One situation where I had to learn something very quickly was during my internship, when I was asked to fine-tune a large language model for a domain-specific task.

, I had never worked hands-on with parameter-efficient fine-tuning methods like LoRA before. The timeline was tight, and I needed to deliver a working prototype within a short period.

To adapt quickly, I first broke the problem into smaller parts.his experience strengthened my ability to learn under pressure, structure complex topics efficiently, and translate theory into practical implementation quickly."""

print(evaluate_answer(question, answer))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
  "clarity": 2,
  "structure": 2,
  "relevance": 2,
  "example_quality": 2,
  "professionalism": 2,
  "total_score": 10,
  "strengths": "Clearly describes a high-pressure learning situation. Demonstrates structured thinking and practical implementation. Mentions a specific technical skill (parameter-efficient fine-tuning).",
  "weaknesses": "Could be more specific about the domain or task. The language model is not named. Could describe the outcome or impact of the prototype."
}

Total Score: 10
Strengths: Describes a high-pressure learning situation, structured thinking, practical implementation, specific technical skill.
Weaknesses: Could be more specific about the domain or task. The language model is not named. Could describe the outcome or impact of the prototype.


In [20]:
!pip install -q openai-whisper
!apt install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 29.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [21]:
import whisper

whisper_model = whisper.load_model("base")  # Changed model to whisper_model
result = whisper_model.transcribe("/content/LLM_Cypher.mp4")

transcript = result["text"]
print(transcript)

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 74.6MiB/s]


 In turn ship hitting the deadlines of threat Got a fine tune in LLM I ain't done this yet They talkin' about Laura, new methods to learn Under pressure I thrive, now it's my turn Yeah broke down the problem in pieces to solve Watchin' my skills in the fire revolve This pressure made a new me, yeah I came through This pressure made a new me, yeah I came through This pressure made a new me, yeah I came through


In [22]:
question = "Tell me about a time you had to learn something quickly."

answer = transcript

print(evaluate_answer(question, answer))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
  "clarity": 0,
  "structure": 0,
  "relevance": 0,
  "example_quality": 0,
  "professionalism": 0,
  "total_score": 0,
  "strengths": "",
  "weaknesses": "Lacks structure, relevance, and quality. Professionalism is non-existent. No STAR elements. Score: 0"
}


In [23]:
!pip install opencv-python mediapipe librosa numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [29]:
import cv2
import numpy as np
import librosa

# --- Ayarlar ---
VIDEO_PATH = "/content/LLM_Cypher.mp4"
FPS = 10
DURATION = 5

# --- Haar Cascade Yüz Tespiti ---
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# --- Video İşleme ---
cap = cv2.VideoCapture(VIDEO_PATH)
video_fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(min(DURATION * video_fps, cap.get(cv2.CAP_PROP_FRAME_COUNT)))
frames_to_process = int(FPS * DURATION)
frame_indices = np.linspace(0, total_frames - 1, frames_to_process).astype(int)

face_detected_frames = 0
mouth_open_values = []

for i in frame_indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        mouth_open_values.append(0)
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) > 0:
        face_detected_frames += 1
        (x, y, w, h) = faces[0]

        # Basit dudak açıklığı tahmini: yüz alt çeyreğini kullan
        mouth_region = gray[y + int(h*0.6): y + h, x: x + w]
        # Dudak açıklığı için yoğunluk farkı
        mouth_open = mouth_region.max() - mouth_region.min()
        mouth_open_values.append(mouth_open)
    else:
        mouth_open_values.append(0)

cap.release()

face_presence_ratio = face_detected_frames / frames_to_process
print("Face Presence Ratio:", face_presence_ratio)

# --- Audio İşleme ---
y, sr = librosa.load(VIDEO_PATH, sr=None, mono=True, offset=0.0, duration=DURATION)
hop_length = max(1, int(len(y) / frames_to_process))
audio_energy = []
for i in range(frames_to_process):
    start = i * hop_length
    end = start + hop_length
    chunk = y[start:end]
    energy = np.sqrt(np.mean(chunk ** 2)) if len(chunk) > 0 else 0
    audio_energy.append(energy)

# --- Lip Sync Skoru ---
correlation = np.corrcoef(audio_energy, mouth_open_values)[0, 1]
print("Lip Sync Correlation:", correlation)

if correlation > 0.6:
    lip_sync_score = 1
elif correlation > 0.3:
    lip_sync_score = 0.5
else:
    lip_sync_score = 0

print("Lip Sync Score:", lip_sync_score)

Face Presence Ratio: 0.0
Lip Sync Correlation: nan
Lip Sync Score: 0


/tmp/ipython-input-2068533974.py:51: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(VIDEO_PATH, sr=None, mono=True, offset=0.0, duration=DURATION)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
